In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf

In [ ]:
from kaggle_datasets import KaggleDatasets

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
import KaggleDatasets()

In [ ]:
!ls

In [ ]:
!ls ../input/ventilator-pressure-prediction

In [ ]:
KaggleDatasets().get_gcs_path('googtrain1')

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(user_credential)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.data import Dataset, TFRecordDataset
from tensorflow.data.experimental import TFRecordWriter

In [ ]:
train_df=pd.read_csv('../input/ventilator-pressure-prediction/train.csv')

In [ ]:
train_df.shape

In [ ]:
# train_df.head()

In [ ]:
# X=np.array(train_df[train_df.columns[1:len(train_df.columns)]])

In [ ]:
# X1=tf.stack(X)

In [ ]:
# ds = Dataset.from_tensor_slices(X1)

In [ ]:
# ds_bytes = ds.map(tf.io.serialize_tensor)

In [ ]:
writer = TFRecordWriter('gs://kds-2c1fe7c24a9609593d2503789706e6f8207fc1d2623f4900e0b28395/train1.tfrecord')

In [ ]:
writer.write(ds_bytes_2)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
ds_bytes_2 = TFRecordDataset('gs://kds-2c1fe7c24a9609593d2503789706e6f8207fc1d2623f4900e0b28395/train1.tfrecord',num_parallel_reads=AUTO)
# ds_2 = ds_bytes_2.map(lambda x: tf.io.parse_tensor(x, out_type=tf.float64))

In [ ]:
ds_bytes_2 = TFRecordDataset('../input/googtrain1/train1.tfrecord',num_parallel_reads=AUTO)

In [ ]:
ds_3 = ds_bytes_2.map(decode)

In [ ]:
def decode(x):
    m=tf.io.parse_tensor(x, out_type=tf.float64)
    return (tf.reshape(m[1:6],(1,5,1)),tf.reshape(m[6],(1,)))

In [ ]:
i=0
for x in ds_3:
    print(x)
    i+=1
    if i==1:
        break

In [ ]:
# m2=tf.compat.v1.data.make_initializable_iterator(ds_2)

In [ ]:
# classifier.save_weights('./modelntrain.h5')

In [ ]:
# epoch=0
# for i in m1:
#     if(epoch>100 and epoch<200):
#         b1=tf.reshape(i[:,0:6],(128,6,1))
#         b2=i[:,6]
#         classifier.fit(b1,b2)
#         epoch+=1
#         print('epochfinish')
#     epoch+=1

In [ ]:
classifier = Sequential()

#Adding the input LSTM network layer
classifier.add(LSTM(128, input_shape=(5,1), return_sequences=True))
classifier.add(Dropout(0.2))
#Adding a second LSTM network layer
classifier.add(LSTM(128))

#Adding a dense hidden layer
classifier.add(Dense(64, activation='relu'))
classifier.add(Dropout(0.2))

#Adding the output layer
classifier.add(Dense(1))
classifier.compile(loss=tf.keras.losses.MeanSquaredError(),optimizer='adam')

In [ ]:
bb=ds_3.batch(64)

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./gweight4.h5',
    save_weights_only=True,
    monitor='loss',
    mode='min',
    verbose=1,save_best_only=True)

In [ ]:
classifier.metrics[0].name

In [ ]:
epochs=0
for x in bb:
    epochs+=1
    print(epochs)
    if epochs>200:
        m1=x[0]
        m2=x[1]
        classifier.fit(m1,m2,callbacks=[model_checkpoint_callback])
        print(epochs)
    if epochs==400:
        break

In [ ]:
classifier.fit(ds_3,epochs=10)

In [ ]:
classifier.fit(ds_3,epochs=100,callbacks=[model_checkpoint_callback],steps_per_epoch=64,batch_size=64)

In [ ]:
tpu_strategy.num_replicas_in_sync

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
with tpu_strategy.scope():
#     load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
#     classifier = tf.keras.models.load_model('../input/modeltry1', options=load_locally) 
    classifier = Sequential()

    #Adding the input LSTM network layer
    classifier.add(LSTM(128, input_shape=(5,1), return_sequences=True))
    classifier.add(Dropout(0.2))
    #Adding a second LSTM network layer
    classifier.add(LSTM(128))

    #Adding a dense hidden layer
    classifier.add(Dense(64, activation='relu'))
    classifier.add(Dropout(0.2))

    #Adding the output layer
    classifier.add(Dense(1))
    opt=tf.keras.optimizers.Adam(learning_rate=0.0001)
    classifier.compile(loss=tf.keras.losses.MeanSquaredError(),optimizer=opt,steps_per_execution=64)

In [ ]:
testdf=pd.read_csv('../input/ventilator-pressure-prediction/test.csv')

In [ ]:
finx=np.array(testdf[testdf.columns[1:len(testdf.columns)]])

In [ ]:
finx=tf.reshape(finx,(finx.shape[0],6,1))

In [ ]:
classifier.load_weights('../input/saved-weights1/modelntrain.h5')

In [ ]:
finpreds=classifier.predict(finx)

In [ ]:
subdf=pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')
subdf.head()

In [ ]:
subdf['pressure']=finpreds

In [ ]:
subdf.to_csv('submission.csv',index=False)

In [ ]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPool2D, Conv3D, Bidirectional, LSTM, GRU, Add
from tensorflow.keras.layers import MaxPool3D, UpSampling2D, GlobalMaxPool2D, GlobalMaxPool3D
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalAveragePooling2D, GlobalAveragePooling3D, Conv2DTranspose, Concatenate
from tensorflow.keras.layers import Dense, Dropout, Activation, Reshape, Flatten, Input

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical, plot_model

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import NASNetMobile, Xception, DenseNet121, MobileNetV2, InceptionV3, InceptionResNetV2, vgg16, resnet50, inception_v3, xception, DenseNet201

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
import math

import sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import jaccard_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import RobustScaler
from scipy import stats


import seaborn as sns

import skimage
from skimage.transform import rotate

from tqdm.notebook import tqdm
from datetime import datetime

from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, roc_auc_score, roc_curve, mean_absolute_error
import numpy as np
import os
import cv2
import pandas as pd
# import imutils
import random
from PIL import Image
import matplotlib.pyplot as plt

import pickle
import urllib

import warnings
warnings.filterwarnings("ignore")

from PIL import Image

import tensorflow_addons as tfa

from IPython.display import HTML

import pydicom
from pydicom.pixel_data_handlers.util import apply_color_lut
import re

import itertools

from sklearn.utils import shuffle

In [ ]:
tpu_strategy.num_replicas_in_sync

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    BATCH_SIZE = tpu_strategy.num_replicas_in_sync * 64
    print("Running on TPU:", tpu.master())
    print(f"Batch Size: {BATCH_SIZE}")
    
except ValueError:
    strategy = tf.distribute.get_strategy()
    BATCH_SIZE = 512
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    print(f"Batch Size: {BATCH_SIZE}")

In [ ]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv').astype(np.float32)
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv').astype(np.float32)